In [1]:
import os
import sys
import torch 
import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

# import log_files
from data_processing import DataProcessing
from text_generation_models import TextGenerationModelFactory

In [2]:
from datasets import load_from_disk

In [3]:
# from huggingface_hub import notebook_login

# notebook_login()

hf_api_key = os.getenv("HUGGING_FACE_TEXT_CLASSIFICATION_API_KEY")

In [4]:
from datasets import load_dataset

imdb = load_dataset("imdb")
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [5]:
imdb["test"][0]

{'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as 

In [6]:
# Get original dataset
data_path = os.path.join(notebook_dir, '../data/')
train_p_o_path = os.path.join(data_path, 'text_classification_tutorial/train_dict.json')
test_p_o_path = os.path.join(data_path, 'text_classification_tutorial/test_dict.json')
files_by_split = {
    'train': str(train_p_o_path),
    'test': str(test_p_o_path)
}
dataset = load_dataset('json', data_files=files_by_split)

dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 2631
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 278
    })
})

In [7]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")

In [8]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [9]:
tokenized_imdb = imdb.map(preprocess_function, batched=True)

In [10]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [11]:
import evaluate

accuracy = evaluate.load("accuracy")

In [12]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [13]:
id2label = {0: "NON-PREDICTION", 1: "PREDICTION"}
label2id = {"NON-PREDICTION": 0, "PREDICTION": 1}

In [14]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert/distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
training_args = TrainingArguments(
    output_dir="my_awesome_model",
    learning_rate=20,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=1,
    weight_decay=1,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_imdb["train"],
    eval_dataset=tokenized_imdb["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/Users/detraviousjamaribrinkley/Documents/Development/research_labs/uf_ds/predictions/.venv_predictions/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,507179716160363256231792786538496.000000,nan,0.500000


TrainOutput(global_step=782, training_loss=3.24283705984887e+32, metrics={'train_runtime': 895.2745, 'train_samples_per_second': 27.924, 'train_steps_per_second': 0.873, 'total_flos': 3310658344060416.0, 'train_loss': 3.24283705984887e+32, 'epoch': 1.0})

In [21]:
text = "This was a masterpiece. Not completely faithful to the books, but enthralling from beginning to end. Might be my favorite of the three."
text = "JPMorgan Chase forecasts that the net profit at Microsoft will decrease in 2024/08/21."

In [22]:
from transformers import pipeline

# model_path = os.path.join(data_path, '')

classifier = pipeline("text-classification", model="my_awesome_model")
classifier(text)

Device set to use mps:0


[{'label': 'NON-PREDICTION', 'score': nan}]

In [23]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("my_awesome_model")
inputs = tokenizer(text, return_tensors="pt")

In [24]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("my_awesome_model")
with torch.no_grad():
    logits = model(**inputs).logits

In [25]:
predicted_class_id = logits.argmax().item()
model.config.id2label[predicted_class_id]

'NON-PREDICTION'